In [64]:
import pandas as pd
import numpy as np
import nltk
import re
import string

import matplotlib.pyplot as plt
%matplotlib inline

In [68]:
# df = pd.read_csv('data/JEOPARDY_CSV.csv', encoding='utf-8')
full_df = pd.read_csv('data/JEOPARDY_CSV.csv', encoding='utf-8')
print(full_df.shape)
full_df.head()

(216930, 7)


,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [69]:
df = full_df.sample(frac=0.1)
df.reset_index(drop=True, inplace=True)

In [70]:
df.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4764,2005-04-28,Double Jeopardy!,CIRCUS SLANG,$2000,"Term for any dessert served in the cookhouse, ...",Abba Dabba
1,3272,1998-11-24,Double Jeopardy!,CLASSICAL MUSIC,$800,"From 1933 to 1935, he served as president of G...",Richard Strauss
2,2987,1997-09-02,Double Jeopardy!,THAT'S RANK!,$200,"As generals of the army, Dwight Eisenhower & O...",Five
3,2130,1993-12-03,Jeopardy!,SCIENCE,$300,Scientist John Dalton suffered from the red-gr...,Colorblindness
4,5957,2010-07-06,Jeopardy!,RHYMES WITH SMART,$400,"Small, slender missile thrown at a board in a ...",a dart


In [71]:
# Remove the dumb spaces
df.columns = ['Show Number', 'Air Date', 'Round', 'Category', 'Value', 'Question', 'Answer']

# Convert to Datetime
df['Air Date'] = pd.to_datetime(df['Air Date'])

# Clean out Value column
df['Value'] = df['Value'].str.replace('$','')
df['Value'] = df['Value'].str.replace(',','')
df['Value'] = df['Value'].apply(lambda x: None if x == 'None' else int(x))
df.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4764,2005-04-28,Double Jeopardy!,CIRCUS SLANG,2000.0,"Term for any dessert served in the cookhouse, ...",Abba Dabba
1,3272,1998-11-24,Double Jeopardy!,CLASSICAL MUSIC,800.0,"From 1933 to 1935, he served as president of G...",Richard Strauss
2,2987,1997-09-02,Double Jeopardy!,THAT'S RANK!,200.0,"As generals of the army, Dwight Eisenhower & O...",Five
3,2130,1993-12-03,Jeopardy!,SCIENCE,300.0,Scientist John Dalton suffered from the red-gr...,Colorblindness
4,5957,2010-07-06,Jeopardy!,RHYMES WITH SMART,400.0,"Small, slender missile thrown at a board in a ...",a dart


In [72]:
df['Question'].value_counts()[0:5]

Preppie runs away from school; spends memorable weekend in New York; gets even more depressed                           2
John Nance Garner                                                                                                       2
John Hoppner, a favorite royal portrait artist of the late 1700s, was rumored to be an illegitimate son of this king    2
Pauling found an "alpha" type of this spiral in proteins; Watson & Crick found a "double" one in DNA                    2
"Talk To The Animals"                                                                                                   2
Name: Question, dtype: int64

In [73]:
df.shape

(21693, 7)

In [74]:
# Drop some useless questions
df = df[df['Question'] != '[audio clue]']
df = df[df['Question'] != '[video clue]']
df = df[df['Question'] != '[filler]']
df = df[df['Question'] != '(audio clue)']

In [75]:
df.shape

(21692, 7)

In [78]:
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')
print(len(stopwords))
print(stopwords[:10])

153
[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your']


In [79]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [80]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [81]:
def create_clean_columns(df):    
    df['clean_question'] = df['Question'].apply(cleanhtml)
    df['clean_answer'] = df['Answer'].apply(cleanhtml)
#     df['clean_category'] = df['Category'].apply(cleanhtml)
    df['everything'] = df['clean_question']+' '+df['clean_answer']#+' '+df['clean_category']
    return df

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    cleantext = cleantext.replace('\n', '')
#     cleantext = cleantext.translate(string.maketrans("",""), string.punctuation)
    cleantext = cleantext.lower()
    return cleantext

In [82]:
create_clean_columns(df)

,Show Number,Air Date,Round,Category,Value,Question,Answer,clean_question,clean_answer,everything
0,4764,2005-04-28,Double Jeopardy!,CIRCUS SLANG,2000.0,"Term for any dessert served in the cookhouse, ...",Abba Dabba,"term for any dessert served in the cookhouse, ...",abba dabba,"term for any dessert served in the cookhouse, ..."
1,3272,1998-11-24,Double Jeopardy!,CLASSICAL MUSIC,800.0,"From 1933 to 1935, he served as president of G...",Richard Strauss,"from 1933 to 1935, he served as president of g...",richard strauss,"from 1933 to 1935, he served as president of g..."
2,2987,1997-09-02,Double Jeopardy!,THAT'S RANK!,200.0,"As generals of the army, Dwight Eisenhower & O...",Five,"as generals of the army, dwight eisenhower & o...",five,"as generals of the army, dwight eisenhower & o..."
3,2130,1993-12-03,Jeopardy!,SCIENCE,300.0,Scientist John Dalton suffered from the red-gr...,Colorblindness,scientist john dalton suffered from the red-gr...,colorblindness,scientist john dalton suffered from the red-gr...
4,5957,2010-07-06,Jeopardy!,RHYMES WITH SMART,400.0,"Small, slender missile thrown at a board in a ...",a dart,"small, slender missile thrown at a board in a ...",a dart,"small, slender missile thrown at a board in a ..."
5,3905,2001-07-20,Double Jeopardy!,ZANY FOR TANZANIA,600.0,"Native to Tanzania, it's the African plant see...",African violet,"native to tanzania, it's the african plant see...",african violet,"native to tanzania, it's the african plant see..."
6,4544,2004-05-13,Double Jeopardy!,"OH, THE THINGS I KNOW!",1600.0,"In 1876 this ""Wild"" West man was killed in the...",Wild Bill Hickok,"in 1876 this ""wild"" west man was killed in the...",wild bill hickok,"in 1876 this ""wild"" west man was killed in the..."
7,3102,1998-02-10,Double Jeopardy!,WE'RE MALAYSIA-BOUND,600.0,"Of proton, electron or neutron, with ""Saga"", i...",Proton,"of proton, electron or neutron, with ""saga"", i...",proton,"of proton, electron or neutron, with ""saga"", i..."
8,5257,2007-06-19,Jeopardy!,AGES AGO,200.0,"When Sumerians used these in war, they had 4 w...",chariots,"when sumerians used these in war, they had 4 w...",chariots,"when sumerians used these in war, they had 4 w..."
9,3859,2001-05-17,Jeopardy!,FIRST NAME'S THE SAME,500.0,"Smith, Rogers, Durant",Will,"smith, rogers, durant",will,"smith, rogers, durant will"


In [83]:
questions = df['everything'].values

In [84]:
#not super pythonic, no, not at all.
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in questions:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)


In [85]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

In [86]:
vocab_frame.head()

,words
term,term
for,for
ani,any
dessert,dessert
serv,served


In [113]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.externals import joblib

# tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
#                                  min_df=0.2, stop_words='english',
#                                  use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

tfidf_vectorizer = TfidfVectorizer(stop_words='english',
                                 tokenizer=tokenize_and_stem, ngram_range=(1,3))

# tfidf_vectorizer = TfidfVectorizer(stop_words='english',
#                                  tokenizer=tokenize_only, ngram_range=(1,3))

# %time tfidf_matrix = tfidf_vectorizer.fit_transform(questions)

tfidf_vectorizer.fit(questions)
joblib.dump(tfidf_vectorizer, 'tfidf_test.pkl')

# print(tfidf_matrix.shape)

['tfidf_test.pkl', 'tfidf_test.pkl_01.npy', 'tfidf_test.pkl_02.npy']

In [114]:
tfidf_vectorizer = joblib.load('tfidf_test.pkl')
tfidf_matrix = tfidf_vectorizer.transform(questions)

In [115]:
terms = tfidf_vectorizer.get_feature_names()

In [116]:
# from sklearn.metrics.pairwise import cosine_similarity
# dist = 1 - cosine_similarity(tfidf_matrix)

In [117]:
from sklearn.cluster import KMeans

num_clusters = 50

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 3min 48s, sys: 12.5 s, total: 4min 1s
Wall time: 2min 10s


In [118]:
from sklearn.externals import joblib

joblib.dump(km,  'doc_cluster.pkl')
km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [119]:
len(clusters)

21692

In [120]:
df['Cluster'] = clusters

In [121]:
df.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer,clean_question,clean_answer,everything,Cluster
0,4764,2005-04-28,Double Jeopardy!,CIRCUS SLANG,2000.0,"Term for any dessert served in the cookhouse, ...",Abba Dabba,"term for any dessert served in the cookhouse, ...",abba dabba,"term for any dessert served in the cookhouse, ...",7
1,3272,1998-11-24,Double Jeopardy!,CLASSICAL MUSIC,800.0,"From 1933 to 1935, he served as president of G...",Richard Strauss,"from 1933 to 1935, he served as president of g...",richard strauss,"from 1933 to 1935, he served as president of g...",6
2,2987,1997-09-02,Double Jeopardy!,THAT'S RANK!,200.0,"As generals of the army, Dwight Eisenhower & O...",Five,"as generals of the army, dwight eisenhower & o...",five,"as generals of the army, dwight eisenhower & o...",26
3,2130,1993-12-03,Jeopardy!,SCIENCE,300.0,Scientist John Dalton suffered from the red-gr...,Colorblindness,scientist john dalton suffered from the red-gr...,colorblindness,scientist john dalton suffered from the red-gr...,5
4,5957,2010-07-06,Jeopardy!,RHYMES WITH SMART,400.0,"Small, slender missile thrown at a board in a ...",a dart,"small, slender missile thrown at a board in a ...",a dart,"small, slender missile thrown at a board in a ...",26


In [122]:
df['Cluster'].value_counts()

26    6796
24    1939
2      639
8      615
4      426
15     403
20     396
47     391
39     391
21     389
35     380
9      376
48     369
5      357
37     341
41     336
11     335
17     331
14     325
3      317
1      316
25     275
40     265
31     248
7      246
0      242
49     237
23     225
33     218
6      218
27     211
10     207
13     204
38     204
29     199
45     197
43     197
46     191
32     188
34     183
42     167
19     154
18     151
16     144
30     140
22     139
44     135
36     121
28     110
12     108
Name: Cluster, dtype: int64

In [123]:
from __future__ import print_function

print("Top terms per cluster:")
print()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    for ind in order_centroids[i, :20]:
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print()
    print()
#     print("Cluster %d categories:" % i, end='')
#     for cat in df.ix[i]['Category']:#.values.tolist():
#         print(' %s,' % cat, end='')
    print()
    print()

Top terms per cluster:

Cluster 0 words: island, rhodes, rhodes, 's, miles, island, national, hawaii, 's, u.s., groups, south, pacific, island, largest, only, hawaiian, indonesia, straits, trinidad,



Cluster 1 words: word, mean, word, 's, latin, precede, greek, 3-letter, latin, greek, used, word, 3-letter, derived, 5-letter, 4-letter, 4-letter, used, comes, 's,



Cluster 2 words: n't, like, said, did, did, ca, ca, 's, does, does, man, wo, wo, know, make, sound, looked, sound, wants, stars,



Cluster 3 words: green, designates, north, went, carolina, stone, north, dark, officially, 's, mountain, emerald, colorful, south, olive, state, american, north, dress, south,



Cluster 4 words: u.s., war, civilization, president, civilization, u.s., during, battle, army, during, 's, u.s., served, u.s., country, only, government, constitutional, american, only,



Cluster 5 words: type, 's, 's, used, like, joint, 'cause, fruit, includes, type, lightly, main, energy, describes, describes, refer